# 💖Fun💖Gut💖

Hi! Welcome to FunGut, the ETH spinoff determined to develop a commercially available platform for users to analyze their gut mycobiome, i.e. fungal microbiome, in order to gain personalized health insights. We recently recieved fecal samples from people living in Europe, Noth Amrtica and Oceania for a pilot trial with the aim of finding relevant factors that affect the gut mycobiome. In this notebook, we will analyze the DNA features that were sequenced from our samples to check if we find any variables that significantly affect either the diversities (both within and between samples) or the differential abundances of features in our samples. Let's begin!

TODO:
    
- Diversity Analsysis nomau outputs aluege und bestimme was signifikant
- Differential analysis dört mache wo in div. ana. relevant

- Denn das beides ou bim kmerizer mache
- Denn vrgliche

# 00 Packages and Directory 📦

Before we do anything, we need to make sure we have all the tools we need to do analyse our data and that we have a system put in place to organize our inputs and outputs. Here we first install geopandas which we use to look at our metadata later on. Subsequently we import all required plugins, set our data directory and create a simple numbered directory to organize our outputs.

In [1]:
pip install geopandas 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import geopandas as gpd

In [3]:
import os
import IPython
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np  

import qiime2 as q2
from qiime2 import Visualization

%matplotlib inline

## 00.01 File directory

In [1]:
! mkdir -p "data/01"
! mkdir -p "data/02"
! mkdir -p "data/03"
! mkdir -p "data/03/denoising_4"
! mkdir -p "data/04"
! mkdir -p "data/05"
! mkdir -p "data/06"
! mkdir -p "data/06/boots-kmer-diversity"
! mkdir -p "data/06/alpha_diversity"
! mkdir -p "data/06/beta_diversity_plots"
! mkdir -p "data/06/permanova"
! mkdir -p "data/07"
! mkdir -p "data/07/ancombc"
! mkdir -p "data/08"
! mkdir -p "data/08/alpha_diversity"
! mkdir -p "data/08/beta_diversity_plots"
! mkdir -p "data/08/permanova"

In [2]:
data_dir = 'data'

# 01 Data import 📥

## 01.01 Importing raw data

Having set up our working environment, lets get started with importing the raw metadata and the raw sequenes. After importing, we cna perform first checks to see what our sequences look like by using qiime tools peek and creating a demux feature table.

In [6]:
!wget "https://polybox.ethz.ch/index.php/s/bLQ6eRWEZo8KmFo/download" -O $data_dir/fungut_metadata.tsv

--2025-12-02 18:53:26--  https://polybox.ethz.ch/index.php/s/bLQ6eRWEZo8KmFo/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18798 (18K) [application/octet-stream]
Saving to: ‘data/fungut_metadata.tsv’

data/fungut_metadat 100%[===================>]  18.36K  --.-KB/s    in 0.008s  

2025-12-02 18:53:26 (2.30 MB/s) - ‘data/fungut_metadata.tsv’ saved [18798/18798]



In [7]:
!wget "https://polybox.ethz.ch/index.php/s/fe7AYe2fBR9jaab/download" -O data/fungut_forward_reads.qza 

--2025-12-02 18:53:27--  https://polybox.ethz.ch/index.php/s/fe7AYe2fBR9jaab/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 712595535 (680M) [application/octet-stream]
Saving to: ‘data/fungut_forward_reads.qza’

data/fungut_forward 100%[===================>] 679.58M   398MB/s    in 1.7s    

2025-12-02 18:53:28 (398 MB/s) - ‘data/fungut_forward_reads.qza’ saved [712595535/712595535]



In [8]:
!qiime tools peek "$data_dir/fungut_forward_reads.qza"

UUID:        3638611d-1767-413b-9390-70ee3d78e4ff
Type:        SampleData[SequencesWithQuality]
Data format: SingleLanePerSampleSingleEndFastqDirFmt


In [9]:
! qiime demux summarize \
    --i-data $data_dir/fungut_forward_reads.qza \
    --o-visualization $data_dir/01/demux_summary.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/01/demux_summary.qzv


In [10]:
Visualization.load(f"{data_dir}/01/demux_summary.qzv")

<visualization: Visualization uuid: f35a848d-ec05-4549-ad4b-c4e1a8a83293>

<div style="background-color: lightblue; padding: 20px;">
Analysing our demultiplexed sequences using the demux feature table, we gather the following information:
</div>

- Our sequences contain a minimum of 20567 foreward reads (FR) per sample
- The mean number of FR is 64221.42 
- The maximum of FR is 113461
- The total number of FR is 9633213
- Quality score for the FR is stable for the entire sequencing length with the median quality score for position 151 (maximum length) being 38 and few medians dipping only slightly below that.


## 01.02 Metadata editing

Having downloaded our metadata file in the previous chapter, we will now need to edit it slightly to ensure we don't run into any errors in the downstream processing. First, we will replace all numeric values that currently are strings of "Not provided" into Nans so we can work with these numbers even though we don't have a set numeric value. For the categorical variables, we want to keep "Not provided" so that we don't loose them in our diversity analysis and differential abundance analysis. Once we have cleaned the "Not provided" strings, we will upload the new metadata file.

In a second step after cleaning the file, we want to add a column for the rural/urban score of each sample based on the coordinates that are already present in the metadata. We do this by downloading our cleaned file, using QGIS and the RUCC (Rural Urban Conrinuum Codes) and import the edited file with the new column that includes the rural/urban score.

Once all that is done, we will also add a new column to group ranges of BMI to their corresponding category on the BMI scale as a last modification of our metadata before we can get to work.

And with that, we have our finalised metadata file (for now...).

In [11]:
metadata_clean = pd.read_csv(f"{data_dir}/fungut_metadata.tsv", sep="\t")

In [12]:
numeric_cols = ["latitude_sample", "longitude_sample", "age_years_sample", "height_cm_sample", "weight_kg_sample", "bmi_sample",] 

for col in numeric_cols:
    if col in metadata_clean.columns:
        metadata_clean[col] = metadata_clean[col].replace(
            ["Not provided"],
            np.nan
        )

In [13]:
metadata_clean.to_csv("data/metadata_clean.tsv", sep="\t", index=False)

#### Rural Urban Continuum Codes

We can now use this cleaned metadata file to add in additional columns to enable us to look at more variables in downstream processing. Firstly, we are interested in not only the geographical coordinates where our samples were taken (already provided) but also in wether or not these locations are rural or urban. As diets and other environmental factors can strongly affect the gut microbiome of our sample hosts, we can later examine if we see any difference between the following three categories: rural, peri-urban and urban.

We add this column using the software QGIS and we apply the RUCC (Rural Urban Continuum Codes) system to attribute one of those three categories to our samples. This step was performed in QGIS directly and thus we need to import the newly modified metadata file again (we needed the NaNs for QGIS, this is why we had to use this order of metadata editing).

In [14]:
!wget "https://polybox.ethz.ch/index.php/s/7XXTnp4in3rYqJQ/download" -O $data_dir/metadata_rucc.tsv

--2025-12-02 18:54:57--  https://polybox.ethz.ch/index.php/s/7XXTnp4in3rYqJQ/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20576 (20K) [application/octet-stream]
Saving to: ‘data/metadata_rucc.tsv’

data/metadata_rucc. 100%[===================>]  20.09K  --.-KB/s    in 0s      

2025-12-02 18:54:57 (43.7 MB/s) - ‘data/metadata_rucc.tsv’ saved [20576/20576]



In [15]:
metadata_rucc = pd.read_csv(f"{data_dir}/metadata_rucc.tsv", sep="\t")

#### BMI Groups

We are further interested in the correlation of BMI with gut microbiome. To better analyze this later on in the pipeline, we create groups of BMI scores based on the official groupings of the National Institute of Health of the USA (most of our samples are from the USA). We do this by creating bins and labels and then applying these to our metadata file.

In [16]:
bins_bmi = [0, 18.5, 24.9, 29.9, 34.9, 39.9, float('inf')]

In [17]:
labels_bmi = ['Underweight', 'Normal', 'Overweight', 'Adipose 1', 'Adipose 2', 'Adipose 3']

In [18]:
metadata_rucc['BMI_category'] = pd.cut(
    metadata_rucc['bmi_sample'],
    bins=bins_bmi,
    labels=labels_bmi,
    right=True
)

In [19]:
metadata_rucc.to_csv("data/metadata_ed.tsv", sep="\t", index=False)

## 01.03 Metadata exploration

Having freshly cleaned and modified our metadata file, let's have a look at what is actually inside and find out more about where our samples come from.

In [20]:
#graphe vo livia

# 02 Trimming primers ✂️

In this chapter we remove the primers from the sequences and then create a new summary table to check if we see any impact on the sequence length distribution (Spoiler alert: We don't).

In [21]:
! qiime cutadapt trim-single \
  --i-demultiplexed-sequences $data_dir/fungut_forward_reads.qza \
  --p-front CTTGGTCATTTAGAGGAAGTAA \
  --o-trimmed-sequences $data_dir/02/fungut_forward_reads_trimmed.qza \
  --verbose

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt -u 0 --error-rate 0.1 --times 1 --overlap 3 --minimum-length 1 -q 0,0 --quality-base 33 --cores 1 -o /tmp/qiime2/jovyan/processes/7501-1764698097.49@jovyan/tmp/q2-OutPath-7gdcsc5k/ERR5327198_01_L001_R1_001.fastq.gz --front CTTGGTCATTTAGAGGAAGTAA /tmp/qiime2/jovyan/data/3638611d-1767-413b-9390-70ee3d78e4ff/data/ERR5327198_01_L001_R1_001.fastq.gz

This is cutadapt 5.1 with Python 3.10.14
Command line parameters: -u 0 -

In [22]:
"""
! qiime demux summarize \
  --i-data $data_dir/02/fungut_forward_reads_trimmed.qza \
  --o-visualization $data_dir/02/demux_summary_posttrimming.qzv
"""

'\n! qiime demux summarize   --i-data $data_dir/02/fungut_forward_reads_trimmed.qza   --o-visualization $data_dir/02/demux_summary_posttrimming.qzv\n'

In [23]:
#Visualization.load(f"{data_dir}/02/demux_summary_posttrimming.qzv")

<div style="background-color: lightblue; padding: 20px;">
Trimming our primers we can see that only XXXXXXXXX of feautres have been edited. Hence, we assume that the primers have already been trimmed as otherwise we would expect many more feautres to have been adjusted.
</div>



# 03 Denoising 🔊 

Perfect! Now we're all set to start working with our demultiplexed and trimmed sequences. We start by denoising the sequences to recieve the holy feature table and the denoised sequences that will be the basis of all downstream processing.

- Weli settings heimer brucht
- Denoising benchmarking vor livia

## 03.01 Denoising benchmarking

(LIVIA SCHRIBE) We tried out several denoising settings to see how they affect both sequence quality and the number of features we keep. To do so we ....

## 03.02 Denoising our demultiplexed sequences using DADA2

<div style="background-color: orange; padding: 20px;">
--p-max-ee 4 — controls the maximum expected errors per read accepted by DADA2’s filtering. 4 is a relatively lenient filter; 1–2 is stricter and will remove more reads. Choose this based on the post-trim quality (look at the demux summary).
<\div>

XXXX why did we use the parameters we used?

In [24]:
"""
#Denoising_1
! qiime dada2 denoise-single \
   --i-demultiplexed-seqs $data_dir/02/fungut_forward_reads_trimmed.qza \
   --p-trim-left 0 \
   --p-trunc-len 0 \
   --p-min-fold-parent-over-abundance 1 \
   --p-max-ee 2 \
    --o-representative-sequences $data_dir/03/denoising_1/dada2_rep_seqs_1.qza \
    --o-table $data_dir/03/denoising_1/dada2_table_1.qza \
    --o-denoising-stats $data_dir/03/denoising_1/dada2_stats_1.qza
"""

'\n#Denoising_1\n! qiime dada2 denoise-single    --i-demultiplexed-seqs $data_dir/02/fungut_forward_reads_trimmed.qza    --p-trim-left 0    --p-trunc-len 0    --p-min-fold-parent-over-abundance 1    --p-max-ee 2     --o-representative-sequences $data_dir/03/denoising_1/dada2_rep_seqs_1.qza     --o-table $data_dir/03/denoising_1/dada2_table_1.qza     --o-denoising-stats $data_dir/03/denoising_1/dada2_stats_1.qza\n'

In [25]:
#Denoising_4

! qiime dada2 denoise-single \
   --i-demultiplexed-seqs $data_dir/02/fungut_forward_reads_trimmed.qza \
   --p-trim-left 0 \
   --p-trunc-len 0 \
   --p-min-fold-parent-over-abundance 4 \
   --p-max-ee 4 \
    --o-representative-sequences $data_dir/03/denoising_4/dada2_rep_seqs_4.qza \
    --o-table $data_dir/03/denoising_4/dada2_table_4.qza \
    --o-denoising-stats $data_dir/03/denoising_4/dada2_stats_4.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: data/03/denoising_4/dada2_table_4.qza
Saved FeatureData[Sequence] to: data/03/denoising_4/dada2_rep_seqs_4.qza
Saved SampleData[DADA2Stats] to: data/03/denoising_4/dada2_stats_4.qza


<div style="background-color: yellow; padding: 20px;">
After denoising, .....
</div>



# 04 Taxonomy 🍄‍🟫 

XXXXXXX
Great! Now that we have a finalised metadata file (for now...) and a denoised feature table, the last thing we need to make all the cool analyses later on is the taxonomic classification of our samples.

To classify our samples, we used a pretrained classifier (INFOS ÜBERE CLASSIFIER) and ran the script below (SCRIPT IRGENDWO IFEèGE WOMER UF EULER BRUCHT HEI) on Euler to recieve a taxonomic classifiaction of our samples. In this notebook, we will first import the output file of the Euler-job and then we will first have a general look at our classification.

In [26]:
!wget "https://polybox.ethz.ch/index.php/s/ybCKaH8Poyz372C/download" -O $data_dir/04/taxjob_euler.qza

--2025-12-02 19:11:42--  https://polybox.ethz.ch/index.php/s/ybCKaH8Poyz372C/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 128973 (126K) [application/octet-stream]
Saving to: ‘data/04/taxjob_euler.qza’

data/04/taxjob_eule 100%[===================>] 125.95K  --.-KB/s    in 0.001s  

2025-12-02 19:11:43 (115 MB/s) - ‘data/04/taxjob_euler.qza’ saved [128973/128973]



## 04.01 Taxonomy overview 

To have a first look at our taxonomic classification, we create the metadata table and the taxa barplots.

In [27]:
! qiime metadata tabulate \
    --m-input-file $data_dir/04/taxjob_euler.qza \
    --o-visualization $data_dir/04/taxonomy_overview.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/04/taxonomy_overview.qzv


In [28]:
Visualization.load(f"{data_dir}/04/taxonomy_overview.qzv")

<visualization: Visualization uuid: 5ac1ecaa-fcfe-40c4-b9ec-4d4731ed7729>

In [29]:
! qiime taxa barplot \
    --i-table $data_dir/03/denoising_4/dada2_table_4.qza \
    --i-taxonomy $data_dir/04/taxjob_euler.qza \
    --m-metadata-file $data_dir/metadata_ed.tsv \
    --o-visualization $data_dir/04/taxa_bar_plots_overview.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/04/taxa_bar_plots_overview.qzv


In [30]:
Visualization.load(f"{data_dir}/04/taxa_bar_plots_overview.qzv")

<visualization: Visualization uuid: 59eeece3-2c0b-4af9-9673-61c978f1567e>

## 04.02 Filtering our data to exclude any non-fungal features

Looking at the taxonomic classification barplots, we see some non-fungal features and some samples with a high degree (>80%) of unclassifiable sequences. Let's have a look at some of these sequences using our metadata table and BLASTing the sequences. Here is an overview of randomly picked features:

| sample ID | BLAST | Reason for filtering |
|-------|-------|-------|
| fed424ac80555019c8816c6efb6f2004   | Fungal endophyte isolate   |  No precise classification   |
| 1ada978c8f3ff08af2668393869257ac    | Rhodotorula mucilaginosa  | Mostly from environmental sources | 
| 201dcc24ff7662a5eb05f31a0d0ca6c0    | no BLAST result found   |      |
| 270002b21dcf309398fe1f945dc902e0    | uncultured fugus   |      |
| 27e5e1e9ffac83de20b1ed2784cecee4    | Rhodotorula mucilaginosa   | Mostly from environmental sources |
| 288c841878718e65bd3f27b674c5b16a    | Mucor racemosus   | Mostly from environmental sources  |
| 31bd8178bf3013d48c62813a535b9e09 | Torulaspora delbrueckii |  Mostly from environmental sources |
| 3e2d3eecabe4355bd15aeab85aebaa83 | Rhodotorula mucilaginosa |  Mostly from environmental sources |
| 478e194c1ec118f79940439cc0c7fd36 | Cladosporium allicinum | plant leaves |
| 4af6fc99c1acf99c1b82d1db3a538c9e | Uncultured fungus | |
| 54e88722f1d30a88eaaf19efc686d249 | Uncultured basidiomycete |  |
| 58de98eb759b336275f22b9a976e97e1 | Rhodotorula mucilaginosa | Mostly from environmental sources |
| 5ad3902302653fe8bb0b938de1d18f68 | Uncultured fungus clone |  |
| 68da1589cb5aa90483bf9ddded795ff5 | Uncultured fungus clone |  |
| 73c2e92790e785ec39b58026ed4010ef | Candida glabrata |k__Fungi;p__Ascomycota;sp__Saccharomycotina;c__Saccharomycetes;o__Saccharomycetales;f__Saccharomycetaceae;g__Candida;s__glabrata
  |
| 818fb915612b44bbbd91b854c738a057 | Fungal endophyte | in plants |
| 97d39fb074a634f6e55e91dd89e69c89 | Blastocystis | parasit |
| a0dc22f3ab283a0afd30e730590fb85c | Uncultured fungus |  |
| a2805006a5012b036fa721982c1324fb | Rhizopus oryzae strain |Mostly from environmental sources|
| afbfe0c693e41ecdd14938a045832793 | Rhodotorula mucilaginosa |Mostly from environmental sources  |
| b2cf5a969f4c7a2f29d49b4216a39244 | Kluyveromyces marxianus | Mostly from environmental sources |
| beb8f130fd78d92d09b9aa1e37fec750 | Escherichia coli | Bacteria |
| c6a89f484b775e3a8a0842ec9d7568e1 | Uncultured fungus |  |
| c84004c0020de7d6ff5bde95677e8688 | Uncultured fungus |  |
| c9da8911059b1b0a4a2980d229a06a69 | Candida albicans  | k__Fungi;p__Ascomycota;sp__Saccharomycotina;c__Saccharomycetes;o__Saccharomycetales;f__Debaryomycetaceae;g__Candida;s__albicans
 |
| d8c7c2823a15d6d0e95a8003cf1cc15b | Rhodotorula mucilaginosa |Mostly from environmental sources  |
| d94d771c3041404138de7df8e52c6cd1 | Aethes rutilana genome assembly | Metazoa |
| ddf643cb96fee9d6ecd44d938b35bd61 | Uncultured fungus  |  |
| eff5b3a232c5c37acde9e8fc390d2c12 | no BLAST result found|  |
| f931ac9a7305cdd99a02c25a624e5bb8 | Pichia kudriavzevii  | Mostly from environmental sources |
| fa340e00a973844b027665f54f7f6fb0 | Rhodotorula mucilaginosa  | Mostly from environmental sources |


As we can see from our randomly sampled unclassified features, almost all of the features stem from environmental sources or are from unclultured fungi. The features belonging to environmental samples and those belonging to plants and parasites are most likely contaminations that could distort downstream analyses. For this reason we decided to filter out any non-fungal features and only keep features that have have been classified to the level of class. We do this by using the qiime taxa filter-table/seqs commands:

In [31]:
! qiime taxa filter-table \
    --i-table $data_dir/03/denoising_4/dada2_table_4.qza \
    --i-taxonomy $data_dir/04/taxjob_euler.qza \
    --p-exclude Protista,Eukaryota_kgd_Incertae_sedis,Ichthyosporia,Metazoa,Viridiplantae \
    --p-include p__ \
    --o-filtered-table $data_dir/04/dada2_table_filtered.qza

! qiime taxa filter-seqs \
    --i-sequences $data_dir/03/denoising_4/dada2_rep_seqs_4.qza \
    --i-taxonomy $data_dir/04/taxjob_euler.qza \
    --p-exclude Protista,Eukaryota_kgd_Incertae_sedis,Ichthyosporia,Metazoa,Viridiplantae \
    --p-include p__ \
    --o-filtered-sequences $data_dir/04/dada2_rep_seqs_filtered.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: data/04/dada2_table_filtered.qza
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[Sequence] to: data/04/dada2_rep_seqs_filtered.qza


In [32]:
! qiime feature-table summarize \
    --i-table $data_dir/04/dada2_table_filtered.qza \
    --m-sample-metadata-file $data_dir/metadata_ed.tsv \
    --o-visualization $data_dir/03/denoising_4/dada2_table_4.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/03/denoising_4/dada2_table_4.qzv


## 04.03 Removing samples high in unclassifiable sequences and high in non-fungi

In our taxonomy barplots, we also saw some samples that are very high (>80%) in unclassified sequences or non-fungal sequences. As they are made up to a large degree of either contaminants or unclassifiable fungi, we want to filter these out to make sure they don't impact our downstream processing. To filter those out, we adjust our metadata and remove the IDs of the unwanted sequences. Then, we create a new feature table summary to see how many samples we lose in the process of filtering.

In [33]:
metadata_ed = pd.read_csv(f"{data_dir}/metadata_ed.tsv", sep="\t")

In [34]:
rem = ["ERR5327575", "ERR5327509", "ERR5327351", "ERR5327544", "ERR5327300", "ERR5327338", "ERR5327529", "ERR5327533", "ERR5327364", "ERR5327535"]

In [35]:
metadata_ed_rem = metadata_ed[~metadata_ed["ID"].isin(rem)]

In [36]:
metadata_ed_rem.to_csv(f"{data_dir}/metadata_ed_rem.tsv", sep="\t", index=False)

In [37]:
!qiime feature-table filter-samples \
    --i-table $data_dir/04/dada2_table_filtered.qza \
    --m-metadata-file $data_dir/metadata_ed_rem.tsv \
    --o-filtered-table $data_dir/04/dada2_table_filtered_samples_removed.qza 

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: data/04/dada2_table_filtered_samples_removed.qza


In [38]:
! qiime feature-table summarize \
  --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
  --o-visualization $data_dir/04/dada2_table_metadata.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/04/dada2_table_metadata.qzv


In [39]:
Visualization.load(f"{data_dir}/04/dada2_table_metadata.qzv")

<visualization: Visualization uuid: be1f4e8f-8ae0-486c-9b9c-f411fa51ab96>

## 04.04 Taxonomy after filtering

In the feautre table summary from the last visualisation, we see that we have now have 140 samples, 10 less than what we started with. Cool, now let's see what has changed in the taxonomic classification after filtering out the unwanted sequences:

In [40]:
!qiime taxa barplot \
    --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
    --i-taxonomy $data_dir/04/taxjob_euler.qza \
    --m-metadata-file $data_dir/metadata_ed_rem.tsv \
    --o-visualization $data_dir/04/taxa_barplots_filtered.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/04/taxa_barplots_filtered.qzv


In [4]:
Visualization.load(f"{data_dir}/04/taxa_barplots_filtered.qzv")

<visualization: Visualization uuid: cde5f7e1-ff96-4882-8d4b-d1a460a5e05f>

# 05 The path to an appropriate sequencing depth 📐

To ensure we comapre our samples fairly (or what the pros call "statistically correct"), we need to standerdize sample sizes by randomly selecting a certain number of sequences from each sample. This certain number is the sequencing depth (SD) and in the following steps, we will dtermine the SD based on the output of alpha rarefaction performed at different max depths.

## 05.01 Finding an appropriate sequencing depth

First we look at the output we get when we do alpha rarefaction at two smartly selected maximal depths (30'000 and 50'000) based on previous trials. For both depths, we see that both the shannon value plateaus very quickly (already at 1000 features for all metadata columns) and the observed features also pleateau early for most metadata columns. Comparing with how many samples we loose (we don't loose any until around 20'000), we decided on a sequencing depth of 20'000 as all curves have plateaued and we have not lost any samples yet.

In [42]:
"""
#30000
#! qiime diversity alpha-rarefaction \
    --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza 
    --p-max-depth 30000 \
    --m-metadata-file $data_dir/metadata_ed_rem.tsv \
    --o-visualization $data_dir/05/fungut_alpha_rarefaction_30000.qzv
"""

'\n#30000\n#! qiime diversity alpha-rarefaction     --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \n    --p-max-depth 30000     --m-metadata-file $data_dir/metadata_ed_rem.tsv     --o-visualization $data_dir/05/fungut_alpha_rarefaction_30000.qzv\n'

In [43]:
"""
#50000
#! qiime diversity alpha-rarefaction \
    --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza 
    --p-max-depth 50000 \
    --m-metadata-file $data_dir/metadata_ed_rem.tsv \
    --o-visualization $data_dir/05/fungut_alpha_rarefaction_50000.qzv
"""

'\n#50000\n#! qiime diversity alpha-rarefaction     --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \n    --p-max-depth 50000     --m-metadata-file $data_dir/metadata_ed_rem.tsv     --o-visualization $data_dir/05/fungut_alpha_rarefaction_50000.qzv\n'

In [44]:
#Visualization.load(f"{data_dir}/05/fungut_alpha_rarefaction_30000.qzv")

In [45]:
#Visualization.load(f"{data_dir}/05/fungut_alpha_rarefaction_50000.qzv")

In the following steps, we check how many samples we would lose fore three possible candidates of sequencing depths by filtering our filtered feature table (so much filtering going on eh?) and summarizing the output to see how many samples we retain.

##### Sequencing depths to test (how many samples do we loose?):
    1. 17500
    2. 20000
    3. 22500
    
Results:
17500: no samples lost (150)
20000: 1 sample lost (149)
22500: 3 samples lost (147)

-> Lets go with 20000. This way we have a good plateau for all curves and lose only 1 sample. 
-> We don't have any extremly low-depth samples that might need filtering (yay!)

In [46]:
"""
#Test = 17500
!qiime feature-table filter-samples \
  --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
  --p-min-frequency 17500 \
  --o-filtered-table $data_dir/05/table_minfreq_17500.qza

!qiime feature-table summarize \
  --i-table $data_dir/05/table_minfreq_17500.qza \
  --m-sample-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/05/table_minfreq_17500_summary.qzv
"""

'\n#Test = 17500\n!qiime feature-table filter-samples   --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza   --p-min-frequency 17500   --o-filtered-table $data_dir/05/table_minfreq_17500.qza\n\n!qiime feature-table summarize   --i-table $data_dir/05/table_minfreq_17500.qza   --m-sample-metadata-file $data_dir/metadata_ed_rem.tsv   --o-visualization $data_dir/05/table_minfreq_17500_summary.qzv\n'

In [47]:
"""
#Depth 20000
!qiime feature-table filter-samples \
  --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
  --p-min-frequency 20000 \
  --o-filtered-table $data_dir/05/table_minfreq_20000.qza

!qiime feature-table summarize \
  --i-table $data_dir/06/table_minfreq_20000.qza \
  --m-sample-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/table_minfreq_20000_summary.qzv
"""

'\n#Depth 20000\n!qiime feature-table filter-samples   --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza   --p-min-frequency 20000   --o-filtered-table $data_dir/05/table_minfreq_20000.qza\n\n!qiime feature-table summarize   --i-table $data_dir/06/table_minfreq_20000.qza   --m-sample-metadata-file $data_dir/metadata_ed_rem.tsv   --o-visualization $data_dir/06/table_minfreq_20000_summary.qzv\n'

In [48]:
"""
#Depth 22500
!qiime feature-table filter-samples \
  --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
  --p-min-frequency 22500 \
  --o-filtered-table $data_dir/05/table_minfreq_22500.qza

!qiime feature-table summarize \
  --i-table $data_dir/05/table_minfreq_22500.qza \
  --m-sample-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/05/table_minfreq_22500_summary.qzv
"""

'\n#Depth 22500\n!qiime feature-table filter-samples   --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza   --p-min-frequency 22500   --o-filtered-table $data_dir/05/table_minfreq_22500.qza\n\n!qiime feature-table summarize   --i-table $data_dir/05/table_minfreq_22500.qza   --m-sample-metadata-file $data_dir/metadata_ed_rem.tsv   --o-visualization $data_dir/05/table_minfreq_22500_summary.qzv\n'

In [49]:
#Visualization.load(f"{data_dir}/05/table_minfreq_17500_summary.qzv")

In [50]:
#Visualization.load(f"{data_dir}/05/table_minfreq_20000_summary.qzv")

In [51]:
#Visualization.load(f"{data_dir}/06/table_minfreq_22500_summary.qzv")

## 05.02 Final check before Bootstrapping

As a last quick check before bootstrapping and kmerizing our features, we perform alpha-rarefaction at a max depth of 23000 50 times to see if the randomness factor of alpha rarefaction might have lead to an unwanted bias. Luckily for us, we see no such effect in the output of this last check so we decided to continue with a SD of 20000.

In [52]:
"""
!qiime diversity alpha-rarefaction \
  --i-table $data_dir/04/dada2_table_filtered.qza \
  --p-max-depth 23000 \
  --p-iterations 50 \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/05/alpha_rarefaction_sanitycheck.qzv
"""

'\n!qiime diversity alpha-rarefaction   --i-table $data_dir/04/dada2_table_filtered.qza   --p-max-depth 23000   --p-iterations 50   --m-metadata-file $data_dir/metadata_ed_rem.tsv   --o-visualization $data_dir/05/alpha_rarefaction_sanitycheck.qzv\n'

In [53]:
#Visualization.load(f"{data_dir}/05/alpha_rarefaction_sanitycheck.qzv")

# 06 Diversity analysis 📊

Aaaalright, now that we have our sequencing depth, we can continue analyzing our data and perform diversity analysis. To do so, we investigate both alpha diversity (=within-sample diversity) and beta diversity (=between sample diversity). 

For the actual computation, we rely on k-mer–based diversity metrics. Recent puplications on ITS workflows show that k-mer methods outperform feature-table approaches by avoiding biases introduced during denoising and taxonomy assignment. In practice, this means we compute all diversity metrics directly from sequence-derived k-mers using qiime boots kmer-diversity. We perform this command at our previously selected sequencing depth and choose a kmer length of 16 after discussing the issue with our lovely TA. As we are running the bootstrapping at 1000 iterations, we must rely on the computational power of euler once again to run the job. Below, we will show the script we provided Euler and then we import the output directory using wget once more. 

## 06.01.01 Bootstrapping and kmerizing our ASVs on Euler 

In [54]:
#We ran the code below on euler to get our bootrstrapped diversity metrics.

"""

#!/bin/bash
#SBATCH --job-name=taxonomy_job
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=128G
#SBATCH --time=24:00:00
#SBATCH --output=taxonomy_output_%j.txt
#SBATCH --mail-type=END


module load stack/2024-06 gcc/12.2.0 python/3.10

source ~/miniconda3/etc/profile.d/conda.sh
conda activate /cluster/scratch/lglauser/conda_envs/q2-boots-amplicon-2025.7
qiime dev refresh-cache

qiime boots kmer-diversity \
  --i-table /cluster/home/lglauser/boot/dada2_table_filtered_samples_removed.qza \
  --i-sequences /cluster/home/lglauser/boot/dada2_rep_seqs_filtered.qza \
  --m-metadata-file /cluster/home/lglauser/boot/metadata_ed_rem.tsv \
  --p-sampling-depth 20000 \
  --p-n 1000 \
  --p-replacement \
  --p-kmer-size 16 \
  --p-alpha-average-method median \
  --p-beta-average-method non-metric-median \
  --p-color-by "country_sample" \
  --output-dir /cluster/scratch/lglauser/boots_kmer_diversity_1000
  
"""

'\n\n#!/bin/bash\n#SBATCH --job-name=taxonomy_job\n#SBATCH --ntasks=1\n#SBATCH --cpus-per-task=1\n#SBATCH --mem-per-cpu=128G\n#SBATCH --time=24:00:00\n#SBATCH --output=taxonomy_output_%j.txt\n#SBATCH --mail-type=END\n\n\nmodule load stack/2024-06 gcc/12.2.0 python/3.10\n\nsource ~/miniconda3/etc/profile.d/conda.sh\nconda activate /cluster/scratch/lglauser/conda_envs/q2-boots-amplicon-2025.7\nqiime dev refresh-cache\n\nqiime boots kmer-diversity   --i-table /cluster/home/lglauser/boot/dada2_table_filtered_samples_removed.qza   --i-sequences /cluster/home/lglauser/boot/dada2_rep_seqs_filtered.qza   --m-metadata-file /cluster/home/lglauser/boot/metadata_ed_rem.tsv   --p-sampling-depth 20000   --p-n 1000   --p-replacement   --p-kmer-size 16   --p-alpha-average-method median   --p-beta-average-method non-metric-median   --p-color-by "country_sample"   --output-dir /cluster/scratch/lglauser/boots_kmer_diversity_1000\n  \n'

## 06.01.02 Importing our Euler results into the notebook

Now that we have run the job on euler, lets download the relevant outputs from the polybox as zip files and unzip them here to create our diversity analysis directory.

In [55]:
!wget "https://polybox.ethz.ch/index.php/s/dZCGqG66fdafKGB/download" -O $data_dir/06/boots-kmer-diversity/alpha_diversities.zip

--2025-12-02 19:16:02--  https://polybox.ethz.ch/index.php/s/dZCGqG66fdafKGB/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 239231768 (228M) [application/zip]
Saving to: ‘data/06/boots-kmer-diversity/alpha_diversities.zip’

data/06/boots-kmer- 100%[===================>] 228.15M   142MB/s    in 1.6s    

2025-12-02 19:16:04 (142 MB/s) - ‘data/06/boots-kmer-diversity/alpha_diversities.zip’ saved [239231768/239231768]



In [56]:
! unzip $data_dir/06/boots-kmer-diversity/alpha_diversities.zip -d $data_dir/06/boots-kmer-diversity/

Archive:  data/06/boots-kmer-diversity/alpha_diversities.zip
   creating: data/06/boots-kmer-diversity/alpha_diversities/
 extracting: data/06/boots-kmer-diversity/alpha_diversities/observed_features.qza  
 extracting: data/06/boots-kmer-diversity/alpha_diversities/pielou_e.qza  
 extracting: data/06/boots-kmer-diversity/alpha_diversities/shannon.qza  


In [57]:
!wget "https://polybox.ethz.ch/index.php/s/Nppkb3p2GdgxjzN/download" -O $data_dir/06/boots-kmer-diversity/distance_matrices.zip

--2025-12-02 19:16:06--  https://polybox.ethz.ch/index.php/s/Nppkb3p2GdgxjzN/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 159582191 (152M) [application/zip]
Saving to: ‘data/06/boots-kmer-diversity/distance_matrices.zip’

data/06/boots-kmer- 100%[===================>] 152.19M   154MB/s    in 1.0s    

2025-12-02 19:16:07 (154 MB/s) - ‘data/06/boots-kmer-diversity/distance_matrices.zip’ saved [159582191/159582191]



In [58]:
! unzip $data_dir/06/boots-kmer-diversity/distance_matrices.zip -d $data_dir/06/boots-kmer-diversity/

Archive:  data/06/boots-kmer-diversity/distance_matrices.zip
   creating: data/06/boots-kmer-diversity/distance_matrices/
 extracting: data/06/boots-kmer-diversity/distance_matrices/braycurtis.qza  
 extracting: data/06/boots-kmer-diversity/distance_matrices/jaccard.qza  


In [59]:
!wget "https://polybox.ethz.ch/index.php/s/arJ6ZLrxYYLJXwR/download" -O $data_dir/06/boots-kmer-diversity/pcoas.zip

--2025-12-02 19:16:09--  https://polybox.ethz.ch/index.php/s/arJ6ZLrxYYLJXwR/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 159578691 (152M) [application/zip]
Saving to: ‘data/06/boots-kmer-diversity/pcoas.zip’

data/06/boots-kmer- 100%[===================>] 152.19M   136MB/s    in 1.1s    

2025-12-02 19:16:11 (136 MB/s) - ‘data/06/boots-kmer-diversity/pcoas.zip’ saved [159578691/159578691]



In [60]:
! unzip $data_dir/06/boots-kmer-diversity/pcoas.zip -d $data_dir/06/boots-kmer-diversity/

Archive:  data/06/boots-kmer-diversity/pcoas.zip
   creating: data/06/boots-kmer-diversity/pcoas/
 extracting: data/06/boots-kmer-diversity/pcoas/.order  
 extracting: data/06/boots-kmer-diversity/pcoas/braycurtis.qza  
 extracting: data/06/boots-kmer-diversity/pcoas/jaccard.qza  


## 06.02 Alpha diversity

### 06.02.01 Making and analyzing the plots

#### Categorical Variables

In [61]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/06/boots-kmer-diversity/alpha_diversities/shannon.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/alpha_diversity/alpha_shannon_group_significance.qzv

!qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/06/boots-kmer-diversity/alpha_diversities/observed_features.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/alpha_diversity/alpha_observed_features_group_significance.qzv

!qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/06/boots-kmer-diversity/alpha_diversities/pielou_e.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/alpha_diversity/alpha_pielou_e_group_significance.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/alpha_diversity/alpha_shannon_group_significance.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/alpha_diversity/alpha_observed_features_group_significance.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_reso

In [5]:
Visualization.load(f"{data_dir}/06/alpha_diversity/alpha_shannon_group_significance.qzv")

<visualization: Visualization uuid: 39109161-7e74-4c1b-8b40-a87b7130f6c9>

In [63]:
Visualization.load(f"{data_dir}/06/alpha_diversity/alpha_observed_features_group_significance.qzv")

<visualization: Visualization uuid: f70a43b2-ab0e-41c7-ac06-b1fd57066827>

In [64]:
Visualization.load(f"{data_dir}/06/alpha_diversity/alpha_pielou_e_group_significance.qzv")

<visualization: Visualization uuid: 324b5c34-681f-425a-810a-de4d2e9132a5>

#### Numeric Variables

In [65]:
!qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/06/boots-kmer-diversity/alpha_diversities/shannon.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/alpha_diversity/alpha_shannon_corrleation.qzv

!qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/06/boots-kmer-diversity/alpha_diversities/observed_features.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/alpha_diversity/alpha_observed_features_correlation.qzv

!qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/06/boots-kmer-diversity/alpha_diversities/pielou_e.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/alpha_diversity/alpha_pielou_e_correlation.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/alpha_diversity/alpha_shannon_corrleation.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/alpha_diversity/alpha_observed_features_correlation.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. Th

In [66]:
Visualization.load(f"{data_dir}/06/alpha_diversity/alpha_shannon_corrleation.qzv")

<visualization: Visualization uuid: 6bd4750c-190f-46f1-a7af-037dccf525be>

In [67]:
Visualization.load(f"{data_dir}/06/alpha_diversity/alpha_observed_features_correlation.qzv")

<visualization: Visualization uuid: 83db5570-166f-4496-b59f-35640183b639>

In [68]:
Visualization.load(f"{data_dir}/06/alpha_diversity/alpha_pielou_e_correlation.qzv")

<visualization: Visualization uuid: f4bcf00f-5e80-4715-a64f-426a30d4b199>

### 06.02.2 Alpha diversity findings

<div style="background-color: yellow; padding: 20px;">
Analysing our alpha diversity plots for all three metrics of shannon, observed features and pielou we found the following things:
</div>

**Numeric variables:**
- Age_years_sample is relevant only for richnenss (pValue=0.0174 in observed_feature alpha correlation)
- All other numeric variables showed no significance for all three metrics

**Categorical Variables:**
- Statistical differnce found for **diet type**: both shannon and observed features show relevant p-Values (Shannon: 0.0273, observed features: 0.0274) for comaprisons to all other diet types (Vegan sample size n=5)
- Statistical differnce found for **IDB status**: Significant p-Value (0.0119) for shannon metric when comparing participants with diagnosed IDB (n=10) to participants without (n=121).

- No statistically relevant differences for the following variables:
    - Country
    - State
    - Sex
    - Celiac disease
    - BMI
    - Rural/Urban/peri

## 06.03 Beta diversity

XXX To investigate between-sample diversity (=beta diversity) we take the beta diversity outputs from the q2 boots kmer diversity directory and convert them to emperor plots. In the plots we can then search for any clustering of our samples (spoiler alert: we don't find any clear clustering).

### 06.03.01 Making the plots

In [69]:
# Bray–Curtis
!qiime emperor plot \
  --i-pcoa $data_dir/06/boots-kmer-diversity/pcoas/braycurtis.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/beta_diversity_plots/kmer_bray_emperor.qzv

# Jaccard
!qiime emperor plot \
  --i-pcoa $data_dir/06/boots-kmer-diversity/pcoas/jaccard.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/beta_diversity_plots/kmer_jaccard_emperor.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/beta_diversity_plots/kmer_bray_emperor.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/beta_diversity_plots/kmer_jaccard_emperor.qzv


In [6]:
Visualization.load(f"{data_dir}/06/beta_diversity_plots/kmer_bray_emperor.qzv")

<visualization: Visualization uuid: 4c3192c0-db35-48d2-870b-6658a2b740f3>

In [71]:
Visualization.load(f"{data_dir}/06/beta_diversity_plots/kmer_jaccard_emperor.qzv")

<visualization: Visualization uuid: d7fc2163-1219-4e8c-ae2d-45532aa7c68e>

### 06.03.02 PERMANOVA

XXXX Having found no clear clustering in our beta diversity plots, let us now check if we find statistically relevant differences between our samples using PERMANOVA. 

Variables of interest:
- Diet type
- IDB Status
- BMI

#### Diet type

In [72]:
# Pairwise Bray-Curtis
!qiime diversity beta-group-significance \
  --i-distance-matrix $data_dir/06/boots-kmer-diversity/distance_matrices/braycurtis.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --m-metadata-column diet_type_sample \
  --p-method permanova \
  --p-permutations 999 \
  --p-pairwise \
  --o-visualization $data_dir/06/permanova/permanova_bray_diet_pairwise.qzv

# Pairwise Jaccard
!qiime diversity beta-group-significance \
  --i-distance-matrix $data_dir/06/boots-kmer-diversity/distance_matrices/jaccard.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --m-metadata-column diet_type_sample \
  --p-method permanova \
  --p-permutations 999 \
  --p-pairwise \
  --o-visualization $data_dir/06/permanova/permanova_jaccard_diet_pairwise.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/permanova/permanova_bray_diet_pairwise.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/permanova/permanova_jaccard_diet_pairwise.qzv


In [73]:
Visualization.load(f"{data_dir}/06/permanova/permanova_bray_diet_pairwise.qzv")

<visualization: Visualization uuid: 856e2168-5ce8-46ca-a4c3-4d358cd172c6>

In [74]:
Visualization.load(f"{data_dir}/06/permanova/permanova_jaccard_diet_pairwise.qzv")

<visualization: Visualization uuid: 6babe64c-5724-4e72-af9d-91ff3d927261>

#### IDB Status

In [75]:
# Pairwise Bray-Curtis
!qiime diversity beta-group-significance \
  --i-distance-matrix $data_dir/06/boots-kmer-diversity/distance_matrices/braycurtis.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --m-metadata-column ibd_sample \
  --p-method permanova \
  --p-permutations 999 \
  --p-pairwise \
  --o-visualization $data_dir/06/permanova/permanova_bray_ibd_pairwise.qzv

# Pairwise Jaccard
!qiime diversity beta-group-significance \
  --i-distance-matrix $data_dir/06/boots-kmer-diversity/distance_matrices/jaccard.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --m-metadata-column ibd_sample \
  --p-method permanova \
  --p-permutations 999 \
  --p-pairwise \
  --o-visualization $data_dir/06/permanova/permanova_jaccard_ibd_pairwise.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/permanova/permanova_bray_ibd_pairwise.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/permanova/permanova_jaccard_ibd_pairwise.qzv


In [76]:
Visualization.load(f"{data_dir}/06/permanova/permanova_bray_ibd_pairwise.qzv")

<visualization: Visualization uuid: e0e47c2b-ea90-41ec-8d26-c48da9586e79>

In [77]:
Visualization.load(f"{data_dir}/06/permanova/permanova_jaccard_ibd_pairwise.qzv")

<visualization: Visualization uuid: dcdb8bd8-8ecf-44d8-99f1-b9cf49e7d75a>

#### BMI 

#### Celiac disease

XXXXXXXXXXXXXXX

Columns for PERMANOVA:
country_sample
sex_sample
diet_type_sample
BMI_category
urban/rural/peri

XXXXXXXXXXXXXX

# 07 Differential Abundance 🧮

## 07.01 Filtering our features

Before we start with the *actual" abundnace analysis, let's prepare for it by improving the resolution of the downstream analysis. We do so by first filtering our feature table to only keep features that appear at least 25 times and in at least 4 samples. Then, we will collapse our taxonomic classification at the species level (=7) so that we can later check and compare the differential abundance both at the ASV level and at the species level.

In [78]:
# FT before filtering
# Summarize feature table (check per-sample read depth, number of features, and overall counts)
! qiime feature-table summarize \
    --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
    --m-sample-metadata-file $data_dir/metadata_ed_rem.tsv \
    --o-visualization $data_dir/07/test_table_abundance_before_filtering.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/test_table_abundance_before_filtering.qzv


In [79]:
Visualization.load(f"{data_dir}/07/test_table_abundance_before_filtering.qzv")

<visualization: Visualization uuid: d600e9f2-5bbe-45b6-9c9d-87b16dc070e5>

In [80]:
! qiime feature-table filter-features \
    --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
    --p-min-frequency 25 \
    --p-min-samples 4 \
    --o-filtered-table $data_dir/07/table_abundance.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: data/07/table_abundance.qza


In [81]:
! qiime feature-table filter-features \
    --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
    --p-min-frequency 20 \
    --p-min-samples 3 \
    --o-filtered-table $data_dir/07/table_abundance_20_3.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: data/07/table_abundance_20_3.qza


In [82]:
# Summarize feature table (check per-sample read depth, number of features, and overall counts)
! qiime feature-table summarize \
    --i-table $data_dir/07/table_abundance.qza \
    --m-sample-metadata-file $data_dir/metadata_ed_rem.tsv \
    --o-visualization $data_dir/07/table_abundance.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/07/table_abundance.qzv


In [83]:
# Summarize feature table (check per-sample read depth, number of features, and overall counts)
! qiime feature-table summarize \
    --i-table $data_dir/07/table_abundance_20_3.qza \
    --m-sample-metadata-file $data_dir/metadata_ed_rem.tsv \
    --o-visualization $data_dir/07/table_abundance_20_3.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/07/table_abundance_20_3.qzv


In [84]:
Visualization.load(f"{data_dir}/07/table_abundance_20_3.qzv")

<visualization: Visualization uuid: dc4e4f2d-6099-45f4-9e31-6c60c6e629b1>

In [85]:
Visualization.load(f"{data_dir}/07/table_abundance.qzv")

<visualization: Visualization uuid: 1f7bf0f6-42e0-4420-82d6-23f8e2fa7e78>

In [86]:
! qiime taxa collapse \
    --i-table $data_dir/07/table_abundance.qza \
    --i-taxonomy $data_dir/04/taxjob_euler.qza \
    --p-level 7 \
    --o-collapsed-table $data_dir/07/table_abundance_collapsed.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: data/07/table_abundance_collapsed.qza


In [87]:
# Summarize feature table (check per-sample read depth, number of features, and overall counts)
! qiime feature-table summarize \
    --i-table $data_dir/07/table_abundance_collapsed.qza \
    --m-sample-metadata-file $data_dir/metadata_ed_rem.tsv \
    --o-visualization $data_dir/07/table_abundance_collapsed.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/07/table_abundance_collapsed.qzv


In [88]:
Visualization.load(f"{data_dir}/07/table_abundance_collapsed.qzv")

<visualization: Visualization uuid: 048d2b6e-da48-49d8-9fba-75d683b3b2fa>

## 07.02 Ancom-BC

Great! Let's now run the actual differnetial analysis, we will run Aconm-BC on the metadata variable of interest on both the ASV level and the species level.

#### Diet type

First for ASV level then also on species level (level 7)

In [89]:
#ASV LEVEL:
# Run ANCOM-BC
! qiime composition ancombc \
    --i-table $data_dir/07/table_abundance.qza \
    --m-metadata-file $data_dir/metadata_ed_rem.tsv \
    --p-formula diet_type_sample \
    --o-differentials $data_dir/07/ancombc/ancombc_diet_differentials.qza

# Generate a barplot of differentially abundant taxa between environments
! qiime composition da-barplot \
    --i-data $data_dir/07/ancombc/ancombc_diet_differentials.qza \
    --o-visualization $data_dir/07/ancombc/ancombc_diet_da_barplot.qzv

# Generate a table of these same values for all taxa
! qiime composition tabulate \
    --i-data $data_dir/07/ancombc/ancombc_diet_differentials.qza \
    --o-visualization $data_dir/07/ancombc/ancombc_diet_results.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[DifferentialAbundance] to: data/07/ancombc/ancombc_diet_differentials.qza
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/07/ancombc/ancombc_diet_da_barplot.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_

In [90]:
#Species LEVEL:
# Run ANCOM-BC
! qiime composition ancombc \
    --i-table $data_dir/07/table_abundance_collapsed.qza \
    --m-metadata-file $data_dir/metadata_ed_rem.tsv \
    --p-formula diet_type_sample \
    --o-differentials $data_dir/07/ancombc/ancombc_diet_differentials_species.qza

# Generate a barplot of differentially abundant taxa between environments
! qiime composition da-barplot \
    --i-data $data_dir/07/ancombc/ancombc_diet_differentials_species.qza \
    --o-visualization $data_dir/07/ancombc/ancombc_diet_da_barplot_species.qzv

# Generate a table of these same values for all taxa
! qiime composition tabulate \
    --i-data $data_dir/07/ancombc/ancombc_diet_differentials_species.qza \
    --o-visualization $data_dir/07/ancombc/ancombc_diet_results_species.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[DifferentialAbundance] to: data/07/ancombc/ancombc_diet_differentials_species.qza
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/07/ancombc/ancombc_diet_da_barplot_species.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resource

##### ASV Level:

In [91]:
Visualization.load(f"{data_dir}/07/ancombc/ancombc_diet_da_barplot.qzv")

<visualization: Visualization uuid: 6f59427b-1654-4f76-84f3-9875b940033f>

In [92]:
Visualization.load(f"{data_dir}/07/ancombc/ancombc_diet_results.qzv")

<visualization: Visualization uuid: 97f1e647-e060-4f1e-8430-ddbf8fa579be>

##### Species Level:

In [93]:
Visualization.load(f"{data_dir}/07/ancombc/ancombc_diet_da_barplot_species.qzv")

<visualization: Visualization uuid: 383c78f1-a8e2-453b-bf57-90dfb0badbbc>

In [94]:
Visualization.load(f"{data_dir}/07/ancombc/ancombc_diet_results_species.qzv")

<visualization: Visualization uuid: afbbef8d-ea49-4a86-892e-81dca47d50db>

# 08 Diversity analysis and differential Abundance WITHOUT using Bootstrapping 🥾🚫

Having performed both diversity analysis and differnetial abundance anylsis on our bootstrapped and kmerized sequences, we now want to do both analyses withouth bootstrapping to accurately compare the two pipelines.

## 08.01 Making the core metrics (kmerizer)

In [15]:
!qiime kmerizer core-metrics \
    --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
    --i-sequences $data_dir/04/dada2_rep_seqs_filtered.qza \
    --m-metadata-file $data_dir/metadata_ed_rem.tsv \
    --p-sampling-depth 20000 \
    --p-kmer-size 16 \
    --output-dir $data_dir/08/kmerizer-results

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: data/08/kmerizer-results/rarefied_table.qza
Saved FeatureTable[Frequency] to: data/08/kmerizer-results/kmer_table.qza
Saved SampleData[AlphaDiversity] to: data/08/kmerizer-results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: data/08/kmerizer-results/shannon_vector.qza
Saved DistanceMatrix to: data/08/kmerizer-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: data/08/kmerizer-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: data/08/kmerizer-results/jaccard_pcoa_results.qza
Saved PCoAResults to: data/08/kmerizer-results/bray_curtis_pcoa_results.qza
Saved Visualization to: data/08/kmerizer

## 08.02 Alpha diversity (kmerizer)

#### Categorical Variables

In [19]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/08/kmerizer-results/shannon_vector.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/08/alpha_diversity/alpha_shannon_group_significance_.qzv

!qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/08/kmerizer-results/observed_features_vector.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/08/alpha_diversity/alpha_observed_features_group_significance.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/08/alpha_diversity/alpha_shannon_group_significance_.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/08/alpha_diversity/alpha_observed_features_group_significance.qzv


In [22]:
Visualization.load(f"{data_dir}/08/alpha_diversity/alpha_shannon_group_significance_.qzv")

<visualization: Visualization uuid: a2731e06-7e64-4dbc-928d-568e886541d7>

In [23]:
Visualization.load(f"{data_dir}/08/alpha_diversity/alpha_observed_features_group_significance.qzv")

<visualization: Visualization uuid: 947f4e63-f89e-4b85-b700-78bb58a234bc>

#### Numeric Variables

In [24]:
!qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/08/kmerizer-results/shannon_vector.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/08/alpha_diversity/alpha_shannon_corrleation.qzv

!qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/08/kmerizer-results/observed_features_vector.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/08/alpha_diversity/alpha_observed_features_correlation.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/08/alpha_diversity/alpha_shannon_corrleation.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/08/alpha_diversity/alpha_observed_features_correlation.qzv


In [25]:
Visualization.load(f"{data_dir}/08/alpha_diversity/alpha_shannon_corrleation.qzv")

<visualization: Visualization uuid: 4dee788e-a3db-4121-aef7-e02effba5d92>

In [26]:
Visualization.load(f"{data_dir}/08/alpha_diversity/alpha_observed_features_correlation.qzv")

<visualization: Visualization uuid: 5ea48a0e-78b6-4421-9486-2f8cba47a684>

### 06.02.2 Alpha diversity findings

<div style="background-color: yellow; padding: 20px;">
Analysing our alpha diversity plots for all three metrics of shannon, observed features and pielou we found the following things:
</div>

**Numeric variables:**
- Age_years_sample is relevant only for richnenss (pValue=0.0174 in observed_feature alpha correlation)
- All other numeric variables showed no significance for all three metrics

**Categorical Variables:**
- Statistical differnce found for **diet type**: both shannon and observed features show relevant p-Values (Shannon: 0.0273, observed features: 0.0274) for comaprisons to all other diet types (Vegan sample size n=5)
- Statistical differnce found for **IDB status**: Significant p-Value (0.0119) for shannon metric when comparing participants with diagnosed IDB (n=10) to participants without (n=121).

- No statistically relevant differences for the following variables:
    - Country
    - State
    - Sex
    - Celiac disease
    - BMI
    - Rural/Urban/peri

## 08.03 Beta diversity (kmerizer)

To investigate between-sample diversity (=beta diversity) we take the beta diversity outputs from the kmerizer directory and convert them to emperor plots. In the plots we can then search for any clustering of our samples (spoiler alert: we don't find any clear clustering here either).

### 08.03.01 Making the plots

In [28]:
# Bray–Curtis
!qiime emperor plot \
  --i-pcoa $data_dir/08/kmerizer-results/bray_curtis_pcoa_results.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/08/beta_diversity_plots/kmer_bray_emperor.qzv

# Jaccard
!qiime emperor plot \
  --i-pcoa $data_dir/08/kmerizer-results/jaccard_pcoa_results.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/08/beta_diversity_plots/kmer_jaccard_emperor.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/08/beta_diversity_plots/kmer_bray_emperor.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/08/beta_diversity_plots/kmer_jaccard_emperor.qzv


In [70]:
Visualization.load(f"{data_dir}/08/beta_diversity_plots/kmer_bray_emperor.qzv")

<visualization: Visualization uuid: 4c3192c0-db35-48d2-870b-6658a2b740f3>

In [29]:
Visualization.load(f"{data_dir}/08/beta_diversity_plots/kmer_jaccard_emperor.qzv")

<visualization: Visualization uuid: f7d6469e-4ce0-4613-a78e-d63ac0cb9e35>

### 08.03.02 PERMANOVA (kmerizer)

XXXX Having found no clear clustering in our beta diversity plots, let us now check if we find statistically relevant differences between our samples using PERMANOVA. 

Variables of interest:
- Diet type
- IDB Status
- BMI

#### Diet type

In [30]:
# Pairwise Bray-Curtis
!qiime diversity beta-group-significance \
  --i-distance-matrix $data_dir/08/kmerizer-results/bray_curtis_distance_matrix.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --m-metadata-column diet_type_sample \
  --p-method permanova \
  --p-permutations 999 \
  --p-pairwise \
  --o-visualization $data_dir/08/permanova/permanova_bray_diet_pairwise.qzv

# Pairwise Jaccard
!qiime diversity beta-group-significance \
  --i-distance-matrix $data_dir/08/kmerizer-results/jaccard_distance_matrix.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --m-metadata-column diet_type_sample \
  --p-method permanova \
  --p-permutations 999 \
  --p-pairwise \
  --o-visualization $data_dir/08/permanova/permanova_jaccard_diet_pairwise.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/08/permanova/permanova_bray_diet_pairwise.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/08/permanova/permanova_jaccard_diet_pairwise.qzv


In [31]:
Visualization.load(f"{data_dir}/08/permanova/permanova_bray_diet_pairwise.qzv")

<visualization: Visualization uuid: d70039d2-18e1-4ba7-a01e-ceb59cfdae33>

In [32]:
Visualization.load(f"{data_dir}/08/permanova/permanova_jaccard_diet_pairwise.qzv")

<visualization: Visualization uuid: 27153ff5-0fbf-4ffa-b035-7a7fdbc7f4e5>

#### IDB Status

In [33]:
# Pairwise Bray-Curtis
!qiime diversity beta-group-significance \
  --i-distance-matrix $data_dir/08/kmerizer-results/bray_curtis_distance_matrix.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --m-metadata-column ibd_sample \
  --p-method permanova \
  --p-permutations 999 \
  --p-pairwise \
  --o-visualization $data_dir/08/permanova/permanova_bray_ibd_pairwise.qzv

# Pairwise Jaccard
!qiime diversity beta-group-significance \
  --i-distance-matrix $data_dir/08/kmerizer-results/jaccard_distance_matrix.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --m-metadata-column ibd_sample \
  --p-method permanova \
  --p-permutations 999 \
  --p-pairwise \
  --o-visualization $data_dir/08/permanova/permanova_jaccard_ibd_pairwise.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/08/permanova/permanova_bray_ibd_pairwise.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/08/permanova/permanova_jaccard_ibd_pairwise.qzv


In [34]:
Visualization.load(f"{data_dir}/08/permanova/permanova_bray_ibd_pairwise.qzv")

<visualization: Visualization uuid: c862a4cb-a838-44f5-8911-7b427573846d>

In [35]:
Visualization.load(f"{data_dir}/08/permanova/permanova_jaccard_ibd_pairwise.qzv")

<visualization: Visualization uuid: e7331e2f-c83e-495c-a9e9-13b0701528e6>

#### BMI 

#### Celiac disease